### Set-up workspace

In [ ]:
!pip install geopandas
!pip install geemap

     |████████████████████████████████| 994 kB 14.8 MB/s 
     |████████████████████████████████| 1.0 MB 43.0 MB/s 
     |████████████████████████████████| 6.6 MB 40.2 MB/s 
     |████████████████████████████████| 464 kB 15.4 MB/s 
     |████████████████████████████████| 1.3 MB 39.9 MB/s 
     |████████████████████████████████| 1.6 MB 38.1 MB/s 
     |████████████████████████████████| 76 kB 9.2 MB/s 
     |████████████████████████████████| 219 kB 21.4 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 147 kB 51.4 MB/s 
     |████████████████████████████████| 133 kB 51.2 MB/s 
     |████████████████████████████████| 76 kB 7.4 MB/s 
     |████████████████████████████████| 1.2 MB 40.6 MB/s 
     |████████████████████████████████| 98 kB 15.4 MB/s 
     |████████████████████████████████| 103 kB 47.7 MB/s 
     |████████████████████████████████| 61 kB 59 kB/s 
     |████████████████████████████████| 95 kB 8.9 MB/s 
     |██████████████████

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import geemap
# geemap.update_package()
import ee

In [ ]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [ ]:
#function to plot dictionaries in python with dropdowns
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### Add map

In [ ]:
Map = geemap.Map()
Map.addLayerControl()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Load in study area

In [ ]:
knp = ee.FeatureCollection('users/jdmwhite/KNP_study_area')
Map.centerObject(knp)
Map.addLayer(knp, {}, 'knp')

### Load in variables

#### Set-up date variables

In [ ]:
startDate = ee.Date('2013-01-01')
endDate = ee.Date('2013-12-01')

In [ ]:
# L7 EVI

# Cloud mask
def cloudMaskL457(image):
  qa = image.select('pixel_qa') ##substitiu a band FMASK
  cloud1 = qa.bitwiseAnd(1<<5).eq(0)
  cloud2 = qa.bitwiseAnd(1<<7).eq(0)
  cloud3 = qa.bitwiseAnd(1<<3).eq(0)

  mask2 = image.mask().reduce(ee.Reducer.min());
  return image.updateMask(cloud1).updateMask(cloud2).updateMask(cloud3).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start"])

# Compute the EVI using an expression.
def evi_calc(image):
    evi = image.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': image.select('B5'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
    }).rename('evi')
    return image.addBands(evi)

l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate(startDate, endDate).map(cloudMaskL457).filterBounds(knp);
l7_evi = l7.map(evi_calc)

#Get information about the bands as a list.
bandNames = l7_evi.first().bandNames()
print('Band names: '+str(bandNames.getInfo())) # ee.List of band names

visParams = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 0.3,
  'gamma': 1.4,
};
# Map.addLayer(l7_evi.median().clip(knp), visParams, 'l7 rgb');
# Map.addLayer(l7_evi.select('evi').mean().clip(knp), {'min': -1, 'max':1, 'palette': ['red','yellow','green']}, 'l7 evi');

evi_l7 = l7_evi.select('evi').mean()

Band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa', 'evi']


In [ ]:
# MODIS EVI
summerStart = ee.Date('2012-11-01')
summerEnd = ee.Date('2013-04-01')
winterStart = ee.Date('2013-05-01')
winterEnd = ee.Date('2013-10-01')

modis_evi_summer = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate(summerStart, summerEnd).select('EVI').mean().multiply(0.0001).clip(knp)
modis_evi_winter = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate(winterStart, winterEnd).select('EVI').mean().multiply(0.0001).clip(knp)

# Map.addLayer(modis_evi_summer, {'min': 0, 'max':1, 'palette': ['yellow','green']}, 'modis evi summer')
# Map.addLayer(modis_evi_winter, {'min': 0, 'max':1, 'palette': ['yellow','green']}, 'modis evi winter')

#### Temperature

In [ ]:
# Temperature from Bioclim
bio_minT = ee.Image("WORLDCLIM/V1/BIO").select(["bio06"]).multiply(0.1).rename('bio_minT').clip(knp)
bio_maxT = ee.Image("WORLDCLIM/V1/BIO").select(["bio05"]).multiply(0.1).rename('bio_maxT').clip(knp)
# Map.addLayer(bio_minT,{'min':5,'max':10, 'palette':['440154', '481567', '482677', '453781', '404788', '39568C', '33638D', '2D708E', '287D8E', '238A8D', '1F968B', '20A387', '29AF7F', '3CBB75', '55C667', '73D055', '95D840', 'B8DE29', 'DCE319', 'FDE725']},'bio_minT')

In [ ]:
# Temperature from TerraClim
ter_minT = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select(['tmmn']).filterDate(startDate,endDate).min().multiply(0.1).rename('ter_minT').clip(knp)
ter_maxT = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select(['tmmx']).filterDate(startDate,endDate).max().multiply(0.1).rename('ter_maxT').clip(knp)
# Map.addLayer(ter_minT,{'min':5,'max':10, 'palette':['440154', '481567', '482677', '453781', '404788', '39568C', '33638D', '2D708E', '287D8E', '238A8D', '1F968B', '20A387', '29AF7F', '3CBB75', '55C667', '73D055', '95D840', 'B8DE29', 'DCE319', 'FDE725']},'ter_minT')

# Note, unless we want to pull out the coldest month value from terraClim, it is probably easiest to use bioclim temps

#### Rainfall

In [ ]:
# Rainfall from Bioclim
bio_annP = ee.Image("WORLDCLIM/V1/BIO").select(["bio12"]).rename('bio_annP').clip(knp)
bio_Pwq = ee.Image("WORLDCLIM/V1/BIO").select(["bio16"]).rename('bio_Pwq').clip(knp)
bio_Pdq = ee.Image("WORLDCLIM/V1/BIO").select(["bio17"]).rename('bio_Pdq').clip(knp)
# Map.addLayer(bio_annP,{'min':300,'max':1000, 'palette':['white','blue']},'bio_annP')

In [ ]:
# Rainfall from TerraClim
ter_annP = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select(['pr']).filterDate(startDate,endDate).sum().rename('ter_annP').clip(knp)
# Map.addLayer(ter_annP,{'min':300,'max':1000, 'palette':['white','blue']},'ter_annP')

# Bioclim has more detail, but seems to have very similar pattern... again probably use bioclim here

#### Topography

In [ ]:
# elevation
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation').clip(knp)
# Map.addLayer(elevation,{'min':0,'max':1000},'elevation')

In [ ]:
# geomorpho90m
cti = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/cti").median().clip(knp)
tri = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tri").median().clip(knp)
slope = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/slope").median().clip(knp)
vrm = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/vrm").median().clip(knp)
roughness = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/roughness").median().clip(knp)
tpi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/tpi").median().clip(knp)
spi = ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/spi").median().clip(knp)
# Map.addLayer(roughness, {}, 'cti')

# See other available vars: https://www.nature.com/articles/s41597-020-0479-6/tables/2

In [ ]:
# microclimate CHILI
chili = ee.Image('CSP/ERGo/1_0/Global/SRTM_CHILI').rename('CHILI').clip(knp)
# Map.addLayer(chili,{},'CHILI')

In [ ]:
# topographic diversity
topD = ee.Image('CSP/ERGo/1_0/Global/SRTM_topoDiversity').rename('topD').clip(knp)
# Map.addLayer(topD,{},'topographic diversity')

#### Soils

In [ ]:
# Soil Grids 250m
bdod_mean = ee.Image("projects/soilgrids-isric/bdod_mean").select('bdod_0-5cm_mean').clip(knp)
cec = ee.Image("projects/soilgrids-isric/cec_mean").select('cec_0-5cm_mean').clip(knp)
fvo = ee.Image("projects/soilgrids-isric/cfvo_mean").select('cfvo_0-5cm_mean').clip(knp)
clay = ee.Image("projects/soilgrids-isric/clay_mean").select('clay_0-5cm_mean').clip(knp)
sand = ee.Image("projects/soilgrids-isric/sand_mean").select('sand_0-5cm_mean').clip(knp)
silt = ee.Image("projects/soilgrids-isric/silt_mean").select('silt_0-5cm_mean').clip(knp)
nitrogen = ee.Image("projects/soilgrids-isric/nitrogen_mean").select('nitrogen_0-5cm_mean').clip(knp)
phh20 = ee.Image("projects/soilgrids-isric/phh20_mean").select('phh20_0-5cm_mean').clip(knp)
soc = ee.Image("projects/soilgrids-isric/soc_mean").select('soc_0-5cm_mean').clip(knp)
ocd = ee.Image("projects/soilgrids-isric/ocd_mean").select('ocd_0-5cm_mean').clip(knp)
ocs = ee.Image("projects/soilgrids-isric/ocs_mean").select('ocs_0-5cm_mean').clip(knp)
# Map.addLayer(nitrogen,{'min':1000,'max':2000, 'palette':['white','red']},'nitrogen')

In [ ]:
# HiHydro Layers 250m
ksat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ksat").first().multiply(0.0001).rename('ksat').clip(knp)
satfield = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/sat-field").first().multiply(0.0001).rename('satfield').clip(knp)
N = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/N").first().multiply(0.0001).rename('N').clip(knp)
alpha = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/alpha").first().multiply(0.0001).rename('alpha').clip(knp)
crit_wilt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/crit-wilt").first().multiply(0.0001).rename('critwilt').clip(knp)
field_cirt = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/field-crit").first().multiply(0.0001).rename('fieldcrit').clip(knp)
ormc = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/ormc").first().multiply(0.0001).rename('ormc').clip(knp)
stc = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/stc").first().multiply(0.0001).rename('stc').clip(knp)
wcavail = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcavail").first().multiply(0.0001).rename('wcavail').clip(knp)
wcpf2 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf2").first().multiply(0.0001).rename('wcpf2').clip(knp)
wcpf3 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf3").first().multiply(0.0001).rename('wcpf3').clip(knp)
wcpf4_2 = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcpf4-2").first().multiply(0.0001).rename('wcpf4-2').clip(knp)
wcres = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcres").first().multiply(0.0001).rename('wcres').clip(knp)
wcsat = ee.ImageCollection("projects/sat-io/open-datasets/HiHydroSoilv2_0/wcsat").first().multiply(0.0001).rename('wcsat').clip(knp)
# Map.addLayer(ksat, {}, 'ksat')

In [ ]:
# Venter SOC 30m
soc_vent = ee.Image('projects/sat-io/open-datasets/NINA/SOC30_SA_mean/SOC_mean_30m_3').divide(100).rename('soc_vent').clip(knp)
# Map.addLayer(soc_vent, {'min': 0, 'max': 16}, 'soc_vent')

In [ ]:
# Hengl Africa Soils 30m
bedrock_depth = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/bedrock_depth").select('b1').rename('H_bed_depth').clip(knp)
bulk_density = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/bulk_density").select('b1').rename('H_bulk_dens').clip(knp)
carbon_organic = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/carbon_organic").select('b1').rename('H_carb_org').clip(knp)
carbon_total = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/carbon_total").select('b1').rename('H_carb_tot').clip(knp)
cation_exchange_capacity = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/cation_exchange_capacity").select('b1').rename('H_cec').clip(knp)
clay_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/clay_content").select('b1').rename('H_clay').clip(knp)
fertility_capability_classification = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/fcc").select('b1').rename('H_fcc').clip(knp)
nitrogen_total = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/nitrogen_total").select('b1').rename('H_N_tot').clip(knp)
ph = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/ph").select('b1').rename('H_ph').clip(knp)
sand_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/sand_content").select('b1').rename('H_sand').clip(knp)
silt_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/silt_content").select('b1').rename('H_silt').clip(knp)
stone_content = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/stone_content").select('b1').rename('H_stone').clip(knp)
texture_class = ee.Image("projects/sat-io/open-datasets/iSDAsoil_Africa_30m/texture_class").select('b1').rename('H_text_class').clip(knp)

# Map.addLayer(bedrock_depth, {}, 'bedrock_depth')

In [ ]:
# TerraClim soil moisture content
ter_smc = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select(['soil']).filterDate(startDate,endDate).mean().multiply(0.1).rename('ter_smc').clip(knp)

Map.addLayer(ter_smc, {}, 'smc')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=35d01260-f96c-43fd-ad32-af4e4f71b35c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>